<a href="https://colab.research.google.com/github/jlcaballero999/tareas_tec/blob/main/coor_2_pols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instlacion de bibliotecas necesarias

In [1]:
!pip install pandas geopandas pyproj shapely

 Importar bibliotecas

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


validar rutas para correr desde drive


In [ ]:
data_path = "/content/drive/My Drive/01_in_coor_csv/NAX_6829594.csv"

Levantar csv

In [ ]:
base = pd.read_csv(data_path, encoding="ISO-8859-1")
base.columns = base.columns.str.strip().str.lower()  # Normalizar nombres de columna
base = base.dropna(subset=['coordenadas'])  # Filtrar filas con coordenadas no nulas

print(base.head())

Crear nombres de columna dinámicos para coordenadas

In [ ]:
max_coords = base['coordenadas'].str.count(',').max() + 1  # Calcular el número máximo de coordenadas
coord_cols = [f'C{i}' for i in range(1, max_coords + 1)]

Separar las coordenadas en columnas individuales

In [ ]:
base[coord_cols] = base['coordenadas'].str.split('|', expand=True)
base['id_consecutivo'] = range(1, len(base) + 1)

print(base.head())

Reorganizar los datos de formato ancho a largo y separar Latitud y Longitud

In [ ]:
base_long = base.melt(
    id_vars=['id_consecutivo'],
    value_vars=coord_cols,
    var_name='coord_num',
    value_name='coords'
).dropna(subset=['coords'])

# Separar Latitud y Longitud
base_long[['latitud', 'longitud']] = base_long['coords'].str.split(',', expand=True)
base_long['latitud'] = base_long['latitud'].str.strip()
base_long['longitud'] = base_long['longitud'].str.strip()

print(base_long.head())

Convertir coordenadas

In [ ]:
def convertir_gms_a_decimal(gms):
    if not isinstance(gms, str):
        return np.nan
    match = re.match(r"(\d+)°(\d+)'(\d+(\.\d+)?)\"?\s?([NSEW])", gms)
    if match:
        grados, minutos, segundos, _, hemisferio = match.groups()
        decimal = float(grados) + float(minutos)/60 + float(segundos)/3600
        if hemisferio in ['S', 'W']:
            return -decimal
        return decimal
    return float(gms) if gms.replace('.', '', 1).isdigit() else np.nan

base_long['latitud'] = base_long['latitud'].apply(convertir_gms_a_decimal)
base_long['longitud'] = base_long['longitud'].apply(convertir_gms_a_decimal)

Convertir a GeoDataFrame ♈

In [ ]:
geometry = gpd.points_from_xy(base_long['longitud'], base_long['latitud'])
data_sf = gpd.GeoDataFrame(base_long, geometry=geometry, crs="EPSG:4326")


Filtrar geometrías válidas y crear polígonos

In [ ]:
data_sf = data_sf.groupby('id_consecutivo').filter(lambda x: x['geometry'].nunique() >= 3)
polygons = data_sf.groupby('id_consecutivo')['geometry'].apply(lambda x: Polygon(x.tolist()))
data_sf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326").reset_index()

Unir atributos originales

In [ ]:
atributos_originales = base.drop(columns=coord_cols)
data_sf = data_sf.merge(atributos_originales, on='id_consecutivo', how='left')

 Guardar  GeoPackage

In [ ]:
output_path = "NAX_6829594.gpkg"
data_sf.to_file(output_path, driver="GPKG")

print("GeoPackage guardado en:", output_path)